<a href="https://colab.research.google.com/github/EVA4-RS-Group/Phase2/blob/master/S7_Variational_AutoEncoders/fileconversion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch==1.5.1+cu101 torchvision==0.6.1+cu101 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 704.4MB 25kB/s 
     |████████████████████████████████| 6.6MB 53kB/s 
  Found existing installation: torch 1.6.0+cu101
    Uninstalling torch-1.6.0+cu101:
      Successfully uninstalled torch-1.6.0+cu101
  Found existing installation: torchvision 0.7.0+cu101
    Uninstalling torchvision-0.7.0+cu101:
      Successfully uninstalled torchvision-0.7.0+cu101


In [2]:
import torch
import io

In [3]:

!unzip data.zip

unzip:  cannot find or open data.zip, data.zip.zip or data.zip.ZIP.


In [4]:
!mkdir samples

In [5]:
import os
import torch
import time
import numpy as np
import matplotlib.pyplot as plt

import torch.utils.data
from torch import nn, optim
from torch.autograd import Variable
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
from torch.distributions.normal import Normal
from torch.distributions import kl_divergence


In [8]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        try:
            nn.init.xavier_uniform_(m.weight.data)
            m.bias.data.fill_(0)
        except AttributeError:
            print("Skipping initialization of ", classname)

In [9]:
class VAE(nn.Module):
    def __init__(self, input_dim, dim, z_dim):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(input_dim, dim, 4, 2, 1),
            nn.BatchNorm2d(dim),
            nn.ReLU(True),
            nn.Conv2d(dim, dim, 4, 2, 1),
            nn.BatchNorm2d(dim),
            nn.ReLU(True),
            nn.Conv2d(dim, dim, 5, 1, 0),
            nn.BatchNorm2d(dim),
            nn.ReLU(True),
            nn.Conv2d(dim, z_dim * 2, 3, 1, 0),
            nn.BatchNorm2d(z_dim * 2)
        )

        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(z_dim, dim, 3, 1, 0),
            nn.BatchNorm2d(dim),
            nn.ReLU(True),
            nn.ConvTranspose2d(dim, dim, 5, 1, 0),
            nn.BatchNorm2d(dim),
            nn.ReLU(True),
            nn.ConvTranspose2d(dim, dim, 4, 2, 1),
            nn.BatchNorm2d(dim),
            nn.ReLU(True),
            nn.ConvTranspose2d(dim, input_dim, 4, 2, 1),
            nn.Tanh()
        )

        self.apply(weights_init)

    def forward(self, x):
        mu, logvar = self.encoder(x).chunk(2, dim=1)

        q_z_x = Normal(mu, logvar.mul(.5).exp())
        p_z = Normal(torch.zeros_like(mu), torch.ones_like(logvar))
        kl_div = kl_divergence(q_z_x, p_z).sum(1).mean()

        x_tilde = self.decoder(q_z_x.rsample())
        return x_tilde, kl_div

In [10]:
DEVICE1 = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
print(DEVICE1)

cuda


In [11]:
INPUT_DIM = 3
DIM = 6
Z_DIM =  8
model = VAE(INPUT_DIM, DIM, Z_DIM)

In [12]:
DEVICE2 = torch.device("cpu")

In [13]:
model.load_state_dict(torch.load('VAE.pt',map_location=DEVICE2))

<All keys matched successfully>

In [16]:
model.eval()
# trace model with a dummy input
traced_model = torch.jit.trace(model, torch.randn(1, 3, 180, 180))
traced_model.save('VAE_jit2.pt')

/usr/local/lib/python3.6/dist-packages/torch/jit/__init__.py:1037: TracerWarning: Trace had nondeterministic nodes. Did you forget call .eval() on your model? Nodes:
	%eps : Float(1, 8, 39, 39) = aten::normal(%283, %293, %294) # /usr/local/lib/python3.6/dist-packages/torch/distributions/utils.py:40:0
This may cause errors in trace checking. To disable trace checking, pass check_trace=False to torch.jit.trace()
  check_tolerance, _force_outplace, True, _module_class)
/usr/local/lib/python3.6/dist-packages/torch/jit/__init__.py:1037: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Not within tolerance rtol=1e-05 atol=1e-05 at input[0, 0, 13, 159] (-0.09480638056993484 vs. -0.25702783465385437) and 93781 other locations (96.48%)
  check_tolerance, _force_outplace, True, _module_class)


In [ ]:
# import torch.onnx

# # Input to the model
# x = torch.randn(1, 3, 180, 180, requires_grad=True)
# torch_out = model(x)

# # Export the model
# torch.onnx.export(model,               # model being run
#                   x,                         # model input (or a tuple for multiple inputs)
#                   "VAE.onnx",   # where to save the model (can be a file or file-like object)
#                   export_params=True,        # store the trained parameter weights inside the model file
#                   opset_version=12,          # the ONNX version to export the model to
#                   do_constant_folding=True,  # whether to execute constant folding for optimization
#                   input_names = ['input'],   # the model's input names
#                   output_names = ['output'], # the model's output names
#                   dynamic_axes={'input' : {0 : 'batch_size'},    # variable lenght axes
#                                 'output' : {0 : 'batch_size'}})

In [15]:
model.save("VAE_jit3.pt" )
torch.jit.save()


AttributeError: ignored

In [ ]:
model = torch.jit.load("VAE_jit2.pt")

In [ ]:
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
from torchvision import datasets, transforms
from torchvision.utils import save_image

In [ ]:
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize(mean=(0.5, 0.5, 0.5),
                                std=(0.5, 0.5, 0.5)) ])

In [ ]:
dataset_test = datasets.ImageFolder('data/test/', transform=transform)
test_loader = DataLoader(dataset=dataset_test, batch_size=16, shuffle=True, drop_last=True, num_workers=4, pin_memory=True)

In [ ]:

def generate_reconstructions():
    model.eval()
    x, _ = test_loader.__iter__().next() 

    # x = x.to(DEVICE)
    # x = x[:16].to(DEVICE)
    out, kl_div = model(x)
    x = (x.data + 1) / 2
    out =  (out.data + 1) / 2
    save_image(
        out,
        'samples/Reconstruction.png',
        nrow=8
    )
    save_image(
        x,
        'samples/sample_input.png',
        nrow=8
    )
generate_reconstructions()